In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
pd.set_option('display.float_format', lambda x: '%.2f' % x)
import numpy as np

import matplotlib.pyplot as plt

import seaborn as sns

from dask_ml.impute import SimpleImputer
import dask.dataframe as dd
from sklearn.preprocessing import LabelEncoder





In [2]:
#df = dd.read_csv('../data/train_sample.csv', dtype={'Census_ProcessorClass':object,'PuaMode':object})

In [3]:
df = pd.read_csv('../data/train_sample.csv', low_memory=False)

In [4]:
df.head()

MachineIdentifier   ProductName EngineVersion  \
0  7b293cc3a0c435ac06d785e84c7ff588  win8defender   1.1.15200.1   
1  91d5b35432343ee710f58bcf752f4645  win8defender   1.1.15200.1   
2  f0e81a93d46e8c32d3a16e4090a2d9fb  win8defender   1.1.15200.1   
3  956cecfe2627e51752e5ffb9c303f9da  win8defender   1.1.15200.1   
4  d6a60da223bf901fb010245872646a10  win8defender   1.1.15200.1   

        AppVersion  AvSigVersion  IsBeta  RtpStateBitfield  IsSxsPassiveMode  \
0  4.18.1807.18075  1.275.1001.0       0              7.00                 0   
1  4.18.1807.18075   1.275.586.0       0              7.00                 0   
2  4.8.10240.17443   1.275.795.0       0              7.00                 0   
3  4.18.1806.18062    1.275.11.0       0              7.00                 0   
4  4.18.1807.18075  1.275.1739.0       0              7.00                 0   

   DefaultBrowsersIdentifier  AVProductStatesIdentifier  ...  \
0                        nan                   53447.00  ...   
1                        nan                   53447.00  ...   
2                        nan                   53447.00  ...   
3                        nan                   53447.00  ...   
4                        nan                   53447.00  ...   

   Census_FirmwareVersionIdentifier  Census_IsSecureBootEnabled  \
0                          33968.00                           0   
1                          33173.00                           1   
2                          72041.00                           0   
3                          33139.00                           1   
4                          56962.00                           0   

   Census_IsWIMBootEnabled  Census_IsVirtualDevice  Census_IsTouchEnabled  \
0                      nan                    0.00                      0   
1                      nan                    0.00                      1   
2                     0.00                    0.00                      0   
3                     0.00                    0.00                      0   
4                      nan                    0.00                      0   

   Census_IsPenCapable  Census_IsAlwaysOnAlwaysConnectedCapable  Wdft_IsGamer  \
0                    0                                     0.00           nan   
1                    1                                     0.00          0.00   
2                    0                                     0.00          0.00   
3                    0                                     0.00          0.00   
4                    0                                     0.00          0.00   

  Wdft_RegionIdentifier HasDetections  
0                   nan             1  
1                  1.00             1  
2                  3.00             1  
3                  1.00             0  
4                 10.00             0  

[5 rows x 83 columns]

In [5]:
df['HasDetections'].value_counts()

0    290032
1    289864
Name: HasDetections, dtype: int64

In [6]:
# look at all stats
# number of charges indication of age?
stats = df.describe().T

In [7]:
stats

count          mean  \
IsBeta                                            579896.00          0.00   
RtpStateBitfield                                  577845.00          6.85   
IsSxsPassiveMode                                  579896.00          0.02   
DefaultBrowsersIdentifier                          28090.00       1660.97   
AVProductStatesIdentifier                         577525.00      47835.53   
AVProductsInstalled                               577525.00          1.33   
AVProductsEnabled                                 577525.00          1.02   
HasTpm                                            579896.00          0.99   
CountryIdentifier                                 579896.00        108.03   
CityIdentifier                                    558826.00      81262.39   
OrganizationIdentifier                            400786.00         24.87   
GeoNameIdentifier                                 579886.00        169.66   
LocaleEnglishNameIdentifier                       579896.00        122.85   
OsBuild                                           579896.00      15722.80   
OsSuite                                           579896.00        575.11   
IsProtected                                       577535.00          0.95   
AutoSampleOptIn                                   579896.00          0.00   
SMode                                             545144.00          0.00   
IeVerIdentifier                                   576059.00        126.59   
Firewall                                          573979.00          0.98   
UacLuaenable                                      579170.00         11.97   
Census_OEMNameIdentifier                          573659.00       2221.44   
Census_OEMModelIdentifier                         573199.00     239032.11   
Census_ProcessorCoreCount                         577161.00          3.99   
Census_ProcessorManufacturerIdentifier            577161.00          4.53   
Census_ProcessorModelIdentifier                   577159.00       2372.27   
Census_PrimaryDiskTotalCapacity                   576400.00     512640.57   
Census_SystemVolumeTotalCapacity                  576402.00     377294.77   
Census_HasOpticalDiskDrive                        579896.00          0.08   
Census_TotalPhysicalRAM                           574577.00       6118.13   
Census_InternalPrimaryDiagonalDisplaySizeInInches 576853.00         16.68   
Census_InternalPrimaryDisplayResolutionHorizontal 576864.00       1547.72   
Census_InternalPrimaryDisplayResolutionVertical   576864.00        897.71   
Census_InternalBatteryNumberOfCharges             562464.00 1125895893.28   
Census_OSBuildNumber                              579896.00      15837.04   
Census_OSBuildRevision                            579896.00        971.30   
Census_OSInstallLanguageIdentifier                575948.00         14.62   
Census_OSUILocaleIdentifier                       579896.00         60.52   
Census_IsPortableOperatingSystem                  579896.00          0.00   
Census_IsFlightingInternal                         97882.00          0.00   
Census_IsFlightsDisabled                          569366.00          0.00   
Census_ThresholdOptIn                             211531.00          0.00   
Census_FirmwareManufacturerIdentifier             567792.00        402.94   
Census_FirmwareVersionIdentifier                  569296.00      33031.09   
Census_IsSecureBootEnabled                        579896.00          0.49   
Census_IsWIMBootEnabled                           212043.00          0.00   
Census_IsVirtualDevice                            578902.00          0.01   
Census_IsTouchEnabled                             579896.00          0.13   
Census_IsPenCapable                               579896.00          0.04   
Census_IsAlwaysOnAlwaysConnectedCapable           575215.00          0.06   
Wdft_IsGamer                                      560212.00          0.28   
Wdft_RegionIdentifier                             

# Let's look at categoricals

In [8]:
cat_variables = stats[(stats['min'] == 0.000) & (stats['max'] == 1.000)]

In [9]:
cat_variables.index

Index(['IsBeta', 'IsSxsPassiveMode', 'HasTpm', 'IsProtected',
       'AutoSampleOptIn', 'SMode', 'Firewall', 'Census_HasOpticalDiskDrive',
       'Census_IsPortableOperatingSystem', 'Census_IsFlightingInternal',
       'Census_IsFlightsDisabled', 'Census_ThresholdOptIn',
       'Census_IsSecureBootEnabled', 'Census_IsVirtualDevice',
       'Census_IsTouchEnabled', 'Census_IsPenCapable',
       'Census_IsAlwaysOnAlwaysConnectedCapable', 'Wdft_IsGamer',
       'HasDetections'],
      dtype='object')

In [10]:
cat_indices = list(cat_variables.index)

In [11]:
# Get rid of NaNs so we can transform floats to ints
imp = SimpleImputer(strategy='most_frequent')
df[cat_indices] = imp.fit_transform(df[cat_indices])

In [12]:
# looking to Kaggle to double-check these are right
# Need to transform the interpreted floats to ints

convert_dict = {}

for index in cat_indices:
    convert_dict[index] = int

In [13]:
df = df.astype(convert_dict)

In [14]:
df.head()

MachineIdentifier   ProductName EngineVersion  \
0  7b293cc3a0c435ac06d785e84c7ff588  win8defender   1.1.15200.1   
1  91d5b35432343ee710f58bcf752f4645  win8defender   1.1.15200.1   
2  f0e81a93d46e8c32d3a16e4090a2d9fb  win8defender   1.1.15200.1   
3  956cecfe2627e51752e5ffb9c303f9da  win8defender   1.1.15200.1   
4  d6a60da223bf901fb010245872646a10  win8defender   1.1.15200.1   

        AppVersion  AvSigVersion  IsBeta  RtpStateBitfield  IsSxsPassiveMode  \
0  4.18.1807.18075  1.275.1001.0       0              7.00                 0   
1  4.18.1807.18075   1.275.586.0       0              7.00                 0   
2  4.8.10240.17443   1.275.795.0       0              7.00                 0   
3  4.18.1806.18062    1.275.11.0       0              7.00                 0   
4  4.18.1807.18075  1.275.1739.0       0              7.00                 0   

   DefaultBrowsersIdentifier  AVProductStatesIdentifier  ...  \
0                        nan                   53447.00  ...   
1                        nan                   53447.00  ...   
2                        nan                   53447.00  ...   
3                        nan                   53447.00  ...   
4                        nan                   53447.00  ...   

   Census_FirmwareVersionIdentifier  Census_IsSecureBootEnabled  \
0                          33968.00                           0   
1                          33173.00                           1   
2                          72041.00                           0   
3                          33139.00                           1   
4                          56962.00                           0   

   Census_IsWIMBootEnabled  Census_IsVirtualDevice  Census_IsTouchEnabled  \
0                      nan                       0                      0   
1                      nan                       0                      1   
2                     0.00                       0                      0   
3                     0.00                       0                      0   
4                      nan                       0                      0   

   Census_IsPenCapable  Census_IsAlwaysOnAlwaysConnectedCapable  Wdft_IsGamer  \
0                    0                                        0             0   
1                    1                                        0             0   
2                    0                                        0             0   
3                    0                                        0             0   
4                    0                                        0             0   

  Wdft_RegionIdentifier HasDetections  
0                   nan             1  
1                  1.00             1  
2                  3.00             1  
3                  1.00             0  
4                 10.00             0  

[5 rows x 83 columns]

# Let's look at "continuous" columns

In [15]:
#Some of these are valid floats or ints but represent categoricals like OsSuite
# Some investigation and decisions made here. In the end we will only have four continuous variables:
# 'Census_InternalBatteryNumberOfCharges', 'Census_SystemVolumeTotalCapacity' 'Census_TotalPhysicalRAM' and
# 'Census_PrimaryDiskTotalCapacity'. The storage media may be a little controversial.
def diff(li1, li2): 
    return (list(set(li1) - set(li2))) 

In [16]:
#Obviously discrete
df['OsSuite'].value_counts()

768    361401
256    217541
272       795
400        57
16         48
305        40
784         9
274         2
49          2
144         1
Name: OsSuite, dtype: int64

In [17]:
#At this point we are guessing it is continuous if not obviously categorial
# The next steps fix this and indeed most cont_variables will be categorical
cont_variables = diff(list(stats.index), cat_indices)

In [18]:
# We will add these to the rest of the categoricals in next steps
not_cont = ['Census_OEMNameIdentifier',
 'AVProductStatesIdentifier',
 'Census_ProcessorModelIdentifier',
 'AVProductsEnabled',
 'OsSuite',
 'IeVerIdentifier',
 'Census_InternalPrimaryDisplayResolutionVertical',
 'Census_OSInstallLanguageIdentifier',
 'Census_OSBuildRevision',
 'DefaultBrowsersIdentifier',
 'CityIdentifier',
 'Census_FirmwareVersionIdentifier',
 'OrganizationIdentifier',
 'Census_OEMModelIdentifier',
 'RtpStateBitfield',
 'Wdft_RegionIdentifier',
 'Census_ProcessorManufacturerIdentifier',
 'AVProductsInstalled',
 'Census_IsWIMBootEnabled',
 'OsBuild',
 'Census_ProcessorCoreCount',
 'Census_FirmwareManufacturerIdentifier',
 'CountryIdentifier',
 'LocaleEnglishNameIdentifier',
 'Census_OSUILocaleIdentifier',
 'Census_InternalPrimaryDiagonalDisplaySizeInInches',
 'GeoNameIdentifier',
 'Census_OSBuildNumber',
 'Census_InternalPrimaryDisplayResolutionHorizontal',
 'UacLuaenable']

In [19]:
new_cont = diff(cont_variables, not_cont)

In [20]:
# Our four actual continuous variables
new_cont

['Census_PrimaryDiskTotalCapacity',
 'Census_SystemVolumeTotalCapacity',
 'Census_InternalBatteryNumberOfCharges',
 'Census_TotalPhysicalRAM']

In [21]:
# Get rid of NaNs and impute with mean
imp = SimpleImputer(strategy='mean')
df[new_cont] = imp.fit_transform(df[new_cont])

# Nominal and Ordinal

In [22]:
nom_and_ord = diff(list(df.columns), list(stats.index))

In [23]:
#del machine identifier from list as we dont want it
del nom_and_ord[22]

In [24]:
all_nom_and_ord_and_cat = nom_and_ord + not_cont

In [25]:
# Fill NaNs
df.fillna(value="NaN", inplace=True)

In [26]:
from dask.distributed import Client
client = Client(n_workers=4, threads_per_worker=4, processes=False, memory_limit='3GB')
client

<Client: 'inproc://192.168.1.7/20172/1' processes=4 threads=16, memory=12.00 GB>

In [27]:
# Impute all_nom_and_ord_and_cat with most_frequent 
imp = SimpleImputer(strategy='most_frequent')
df[all_nom_and_ord_and_cat] = imp.fit_transform(df[all_nom_and_ord_and_cat])

In [28]:
df.head()

MachineIdentifier   ProductName EngineVersion  \
0  7b293cc3a0c435ac06d785e84c7ff588  win8defender   1.1.15200.1   
1  91d5b35432343ee710f58bcf752f4645  win8defender   1.1.15200.1   
2  f0e81a93d46e8c32d3a16e4090a2d9fb  win8defender   1.1.15200.1   
3  956cecfe2627e51752e5ffb9c303f9da  win8defender   1.1.15200.1   
4  d6a60da223bf901fb010245872646a10  win8defender   1.1.15200.1   

        AppVersion  AvSigVersion  IsBeta RtpStateBitfield  IsSxsPassiveMode  \
0  4.18.1807.18075  1.275.1001.0       0             7.00                 0   
1  4.18.1807.18075   1.275.586.0       0             7.00                 0   
2  4.8.10240.17443   1.275.795.0       0             7.00                 0   
3  4.18.1806.18062    1.275.11.0       0             7.00                 0   
4  4.18.1807.18075  1.275.1739.0       0             7.00                 0   

  DefaultBrowsersIdentifier AVProductStatesIdentifier  ...  \
0                       NaN                  53447.00  ...   
1                       NaN                  53447.00  ...   
2                       NaN                  53447.00  ...   
3                       NaN                  53447.00  ...   
4                       NaN                  53447.00  ...   

  Census_FirmwareVersionIdentifier Census_IsSecureBootEnabled  \
0                         33968.00                          0   
1                         33173.00                          1   
2                         72041.00                          0   
3                         33139.00                          1   
4                         56962.00                          0   

   Census_IsWIMBootEnabled  Census_IsVirtualDevice Census_IsTouchEnabled  \
0                      NaN                       0                     0   
1                      NaN                       0                     1   
2                     0.00                       0                     0   
3                     0.00                       0                     0   
4                      NaN                       0                     0   

  Census_IsPenCapable Census_IsAlwaysOnAlwaysConnectedCapable  Wdft_IsGamer  \
0                   0                                       0             0   
1                   1                                       0             0   
2                   0                                       0             0   
3                   0                                       0             0   
4                   0                                       0             0   

  Wdft_RegionIdentifier HasDetections  
0                   NaN             1  
1                  1.00             1  
2                  3.00             1  
3                  1.00             0  
4                 10.00             0  

[5 rows x 83 columns]

In [29]:
df.to_csv('../data/train_sample_processed.csv', index=False)

In [30]:
all_nom_and_ord_and_cat

['SmartScreen',
 'Census_GenuineStateName',
 'Census_OSWUAutoUpdateOptionsName',
 'Census_ChassisTypeName',
 'Census_ActivationChannel',
 'Census_PowerPlatformRoleName',
 'AppVersion',
 'SkuEdition',
 'MachineIdentifier',
 'Census_OSEdition',
 'Census_OSSkuName',
 'Census_InternalBatteryType',
 'Census_ProcessorClass',
 'Census_OSInstallTypeName',
 'AvSigVersion',
 'Census_MDC2FormFactor',
 'Census_PrimaryDiskTypeName',
 'EngineVersion',
 'OsVer',
 'Processor',
 'ProductName',
 'Platform',
 'Census_FlightRing',
 'OsPlatformSubRelease',
 'PuaMode',
 'Census_OSBranch',
 'OsBuildLab',
 'Census_OSVersion',
 'Census_OSArchitecture',
 'Census_OEMNameIdentifier',
 'AVProductStatesIdentifier',
 'Census_ProcessorModelIdentifier',
 'AVProductsEnabled',
 'OsSuite',
 'IeVerIdentifier',
 'Census_InternalPrimaryDisplayResolutionVertical',
 'Census_OSInstallLanguageIdentifier',
 'Census_OSBuildRevision',
 'DefaultBrowsersIdentifier',
 'CityIdentifier',
 'Census_FirmwareVersionIdentifier',
 'Organizat

In [31]:
df['Census_PrimaryDiskTotalCapacity'  ].value_counts()

476940.00    184120
953869.00    141640
305245.00     31027
122104.00     30594
244198.00     29490
              ...  
66048.00          1
66000.00          1
262844.00         1
475615.00         1
68902.00          1
Name: Census_PrimaryDiskTotalCapacity, Length: 1195, dtype: int64